In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/ex_comparison_forecasting

Mounted at /content/drive
/content/drive/MyDrive/ex_comparison_forecasting


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import norm
import math
import sklearn.preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
import pickle
from scipy import interpolate
import random
import time
from lightgbm import LGBMRegressor
# import optuna
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from scipy.stats import linregress

import argparse
import torch
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import r2_score as R2


In [ ]:
with open(f'list_EQ_id.p', 'rb') as f:
    list_EQid = pickle.load(f)


In [ ]:
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P


In [ ]:
def prepare_data_batch(x, N1, N2, list_output_time, j = 2):
  N0 = np.shape(x)[1]
  xxx = np.stack([x[:,i:N0 - N1+i+1:j, :] for i in range(N1)], axis=2)
  y = xxx[:,:,-N2:,-len(list_output_time):]

  return np.array(xxx), np.array(y)


In [ ]:
def data_provider2(data_time, data_static, data_y, batch_size=128):
    data_set = h_Dataset(data_time, data_static, data_y)
    data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True, drop_last=True)
    return data_set, data_loader


In [ ]:
class h_Dataset(Dataset):
    def __init__(self, data_time, data_static, data_y):
        self.data_time = data_time
        self.data_static = data_static
        self.data_y = data_y

    def __getitem__(self, index):
        seq_time = self.data_time[index]
        seq_static = self.data_static[index]
        seq_y = self.data_y[index]
        return seq_time, seq_static, seq_y

    def __len__(self):
        return len(self.data_y)



In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model, kernel_size=3, padding=1, padding_mode='circular', bias=False)

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x

class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, dropout=0.1):
        super(DataEmbedding, self).__init__()
        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
          x = self.value_embedding(x) + self.position_embedding(x)
          return self.dropout(x)

class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()

        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False
        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


In [ ]:
class Data_aggregation(nn.Module):
    def __init__(self, L_all, N_static, dropout=0.1):
        super(Data_aggregation, self).__init__()
        self.projection = nn.Sequential(nn.Linear(N_static, L_all), nn.ReLU(), nn.Linear(L_all, L_all))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x_time, x_static):
        x = self.projection(x_static)
        x = self.dropout(x)
        x = x.permute(0, 2, 1)
        x = torch.cat([x_time, x], dim=2)

        return x



In [ ]:
def gen_data_AI_batch_2_model(df_new, list_idx, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler):
  L_in = L_all - L_out
  signal = [np.stack(list(df_new[k][list_idx])) for k in list_time_series ]
  signal = np.stack(signal).transpose(1,2,0)
  signal = np.pad(signal,((0,0),(L_all,0),(0,0)),constant_values=0.0)
  signal = signal.reshape(-1, N_input + N_output)
  scaled_signal = real_scaler.transform(signal)
  scaled_signal = scaled_signal.reshape(-1, L_ground+L_all, N_input + N_output)
  X_times_series, Y_label = prepare_data_batch(scaled_signal, L_all, L_out, list_output_time, j = sliding_step)
  X_times_series[:,:,L_in:,N_input:N_input+N_output] = 0.0

  static = np.stack(list(df_new[k][list_idx] for k in list_static)).T
  scaled_static = static_scaler.transform(static)
  X_static = scaled_static.reshape(-1,1,1,N_static)
  X_static = np.tile(X_static, [1, X_times_series.shape[1], 1, 1])
  X_times_series = X_times_series.reshape(-1, L_all, N_input+N_output)
  X_static = X_static.reshape(-1, 1, N_static)
  Y_label =  Y_label.reshape(-1, L_out, N_output)

  return X_times_series, X_static, Y_label


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, configs, L_all, L_out, N_input, N_output):
        super(ResBlock, self).__init__()

        self.temporal = nn.Sequential(
            nn.Linear(L_all, configs.d_model),
            nn.ReLU(),
            nn.Linear(configs.d_model, L_all),
            nn.Dropout(configs.dropout)
        )

        self.channel = nn.Sequential(
            nn.Linear(N_input+N_output+1, configs.d_model),
            nn.ReLU(),
            nn.Linear(configs.d_model, N_input+N_output+1),
            nn.Dropout(configs.dropout)
        )

    def forward(self, x):
        # x: [B, L, D]
        x = x + self.temporal(x.transpose(1, 2)).transpose(1, 2)
        x = x + self.channel(x)

        return x


class TSMixer_dis(nn.Module):
    def __init__(self, configs, L_all, L_out, N_input, N_output):
        super(TSMixer_dis, self).__init__()
        self.layer = configs.e_layers
        self.model = nn.ModuleList([ResBlock(configs, L_all, L_out, N_input, N_output) for _ in range(configs.e_layers)])
        self.projection = nn.Sequential(nn.Linear(L_all, L_out), nn.ReLU())
        self.out_layer = nn.ModuleList([nn.Linear(N_input+N_output+1, N_output, bias=False) for i in range(7)])

    def forward(self, x_enc):
        x = x_enc
        for i in range(self.layer):
            x = self.model[i](x)

        x = self.projection(x.permute(0, 2, 1)).permute(0, 2, 1)
        x_out = torch.stack([self.out_layer[i](x) for i in range(7)], dim=-1)

        return x_out

class model_parameters:
  def __init__(self):
    self.d_model = 128
    self.e_layers = 2
    self.dropout = 0.1

# [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98],
class MAE_2():
    def loss(y_pred, target):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        L = torch.zeros(1).to(device)
        for i, q in enumerate([0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]):
          errors = y_pred[:,:,:,i] - target
          L += torch.mean(torch.max((q - 1) * errors, q * errors))

        return L


In [ ]:
def one_function_optim(d_model, e_layers, L_in, L_out, dropout):

    L_all = L_in + L_out
    sliding_step = L_out

    with open(f'EQall.p','rb') as f:
      EQall = pickle.load(f)

    print(ex_name)

    with open(f'new_p_data_allEQ_{ex_name}.p', 'rb') as f:
      df_new = pickle.load(f)

    for i in range(len(df_new)):
      L = len(df_new['disp_ground'][i])
      if L >3000:
        df_new['disp_ground'][i] = df_new['disp_ground'][i][:3000]
      else:
        df_new['disp_ground'][i] = list(df_new['disp_ground'][i]) + [0]*(3000-L)

    list_output_time = []
    list_static = []

    for k in list(df_new):
      if (str(k)[:4] == 'disp') & (str(k)[-1] != 'd'):
        list_output_time.append(k)

      if str(k)[:4] != 'disp':
        list_static.append(k)

    df_new['EQid'] = list_EQid
    list_len = np.array([len(df_new['disp_2'][ip]) for ip in range(len(df_new))])
    list_idx = np.where(list_len!=3000)[0]
    df_new = df_new.drop(index=list_idx)
    df_new = df_new.reset_index()


    L_signal_input = len(df_new['disp_ground'][0])
    L_ground = L_signal_input

    Ndim_encoding = 2
    P_encoding = getPositionEncoding(seq_len=L_signal_input, d=Ndim_encoding, n=10000)
    for d in range(Ndim_encoding):
      df_new[f'encode_{d}'] = [P_encoding[:,d]]*len(df_new)

    list_input_time = ['disp_ground']+[f'encode_{d}' for d in range(Ndim_encoding)]
    list_time_series = list_input_time + list_output_time
    L_in = L_all-L_out
    N_input= len(list_input_time)
    N_output= len(list_output_time)
    N_static = len(list_static)
    N_all = N_input+N_output+N_static
    N_time_series = len(list_input_time)+len(list_output_time)+1

    static_scaler = sklearn.preprocessing.StandardScaler().fit(df_new[list_static].values)
    target = np.array([df_new[i][0] for i in list_output_time]).T
    target_scaler = sklearn.preprocessing.StandardScaler().fit(target)
    real = np.array([df_new[i][0] for i in list_time_series]).T
    real_scaler = sklearn.preprocessing.StandardScaler().fit(real)


    train_time, train_static, train_label = gen_data_AI_batch_2_model(df_new, idx_train, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    val_time, val_static, val_label = gen_data_AI_batch_2_model(df_new, idx_valid, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    _, train_loader = data_provider2(train_time, train_static, train_label, batch_size=64)
    _, valid_loader = data_provider2(val_time, val_static, val_label, batch_size=64)


    args = model_parameters()
    args.d_model = d_model
    args.e_layers = e_layers
    args.dropout = dropout

    model_tor = TSMixer_dis(args, L_all, L_out, N_input, N_output)

    N_epoch = 250
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    aggregation = Data_aggregation(L_all, N_static, dropout)
    aggregation = aggregation.float().to(device)
    model_tor = model_tor.float().to(device)
    model_optim = torch.optim.Adam(list(model_tor.parameters())+list(aggregation.parameters()), lr=0.001)
    criterion = MAE_2

    for epoch in range(N_epoch):
        train_loss = []
        for batch_time, batch_static, batch_y in train_loader:
            model_optim.zero_grad()
            batch_time, batch_static, batch_y = batch_time.float().to(device), batch_static.float().to(device), batch_y.float().to(device)
            batch_x = aggregation(batch_time, batch_static)
            outputs = model_tor(batch_x)
            loss = criterion.loss(outputs, batch_y)
            loss.backward()
            train_loss.append(loss.item())
            model_optim.step()


    list_idx = idx_valid
    x_time, x_static, _ = gen_data_AI_batch_2_model(df_new, list_idx, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    x_time = x_time.reshape(len(list_idx),-1,L_all, N_input+N_output)
    x_static = x_static.reshape(len(list_idx),-1, 1, N_static)

    y_predict_all = np.zeros((len(x_time), L_ground+L_all, N_output))

    for j in range(int(L_ground/L_out)+1):
      x_time[:,j,:L_all, N_input:N_input+N_output] = y_predict_all[:,L_out*j:L_all+L_out*j,:]
      xt = torch.from_numpy(x_time[:,j,:,:]).float().to('cuda')
      xs = torch.from_numpy(x_static[:,j,:,:]).float().to('cuda')
      batch_x = aggregation(xt, xs)
      y_pred_all = model_tor(batch_x).to('cuda').cpu().detach().numpy()
      y_predict_all[:,L_in+L_out*j:L_all+L_out*j,:] = y_pred_all[:,:,:,3]

    y_predict_all = target_scaler.inverse_transform(y_predict_all.reshape(-1, N_output))
    y_predict_all = y_predict_all.reshape(len(x_time), -1, N_output)


    dict_MAE = {k:[] for k in list_output_time}
    dict_MAPE = {k:[] for k in list_output_time}
    dict_RMSE = {k:[] for k in list_output_time}
    dict_R2 = {k:[] for k in list_output_time}

    for i_s in range(len(list_output_time)):
      for i in range(len(list_idx)):
        calMAE = MAE(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_MAE[list_output_time[i_s]].append(calMAE)
        calR2 = R2(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_R2[list_output_time[i_s]].append(calR2)

        calMAPE = MAPE(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_MAPE[list_output_time[i_s]].append(calMAPE)
        calRMSE = RMSE(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_RMSE[list_output_time[i_s]].append(calRMSE)



    mean_MAE = [np.mean(dict_MAE[k]) for k in dict_MAE]
    mean_MAPE = [np.mean(dict_MAPE[k]) for k in dict_MAPE]
    mean_RMSE = [np.mean(dict_RMSE[k]) for k in dict_RMSE]
    mean_R2 = [np.mean(dict_R2[k]) for k in dict_R2]

    return mean_R2[-1], mean_MAE[-1], mean_MAPE[-1], mean_RMSE[-1]


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.1 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
def one_function(trial):
    N_embed = trial.suggest_categorical("d_model", [32, 64, 128, 256, 512])
    e_layers= trial.suggest_int("e_layers", 2,5)
    L_in   = trial.suggest_categorical("L_in", [50, 100, 150, 250, 500])
    L_out   = trial.suggest_categorical("L_out", [100, 300, 500, 750, 1000])
    dropout   = trial.suggest_categorical("dropout", [0.1, 0.2, 0.3])
    L_all = L_in + L_out
    sliding_step = L_out

    with open(f'EQall.p','rb') as f:
      EQall = pickle.load(f)

    with open(f'new_p_data_allEQ_{ex_name}.p', 'rb') as f:
      df_new = pickle.load(f)

    for i in range(len(df_new)):
      L = len(df_new['disp_ground'][i])
      if L >3000:
        df_new['disp_ground'][i] = df_new['disp_ground'][i][:3000]
      else:
        df_new['disp_ground'][i] = list(df_new['disp_ground'][i]) + [0]*(3000-L)

    list_output_time = []
    list_static = []

    for k in list(df_new):
      if (str(k)[:4] == 'disp') & (str(k)[-1] != 'd'):
        list_output_time.append(k)

      if str(k)[:4] != 'disp':
        list_static.append(k)

    df_new['EQid'] = list_EQid
    list_len = np.array([len(df_new['disp_2'][ip]) for ip in range(len(df_new))])
    list_idx = np.where(list_len!=3000)[0]
    df_new = df_new.drop(index=list_idx)
    df_new = df_new.reset_index()

    print(ex_name, len(df_new))

    L_signal_input = len(df_new['disp_ground'][0])
    L_ground = L_signal_input

    Ndim_encoding = 2
    P_encoding = getPositionEncoding(seq_len=L_signal_input, d=Ndim_encoding, n=10000)
    for d in range(Ndim_encoding):
      df_new[f'encode_{d}'] = [P_encoding[:,d]]*len(df_new)

    list_input_time = ['disp_ground']+[f'encode_{d}' for d in range(Ndim_encoding)]
    list_time_series = list_input_time + list_output_time
    L_in = L_all-L_out
    N_input= len(list_input_time)
    N_output= len(list_output_time)
    N_static = len(list_static)
    N_all = N_input+N_output+N_static
    N_time_series = len(list_input_time)+len(list_output_time)+1

    static_scaler = sklearn.preprocessing.StandardScaler().fit(df_new[list_static].values)
    target = np.array([df_new[i][0] for i in list_output_time]).T
    target_scaler = sklearn.preprocessing.StandardScaler().fit(target)
    real = np.array([df_new[i][0] for i in list_time_series]).T
    real_scaler = sklearn.preprocessing.StandardScaler().fit(real)


    train_time, train_static, train_label = gen_data_AI_batch_2_model(df_new, idx_train, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    val_time, val_static, val_label = gen_data_AI_batch_2_model(df_new, idx_valid, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    _, train_loader = data_provider2(train_time, train_static, train_label, batch_size=64)
    _, valid_loader = data_provider2(val_time, val_static, val_label, batch_size=64)


    args = model_parameters()
    args.d_model = d_model
    args.e_layers = e_layers
    args.dropout = dropout

    model_tor = TSMixer_dis(args, L_all, L_out, N_input, N_output)

    N_epoch = 200
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    aggregation = Data_aggregation(L_all, N_static, dropout)
    aggregation = aggregation.float().to(device)
    model_tor = model_tor.float().to(device)
    model_optim = torch.optim.Adam(list(model_tor.parameters())+list(aggregation.parameters()), lr=0.001)
    criterion = MAE_2

    for epoch in range(N_epoch):
        train_loss = []
        for batch_time, batch_static, batch_y in train_loader:
            model_optim.zero_grad()
            batch_time, batch_static, batch_y = batch_time.float().to(device), batch_static.float().to(device), batch_y.float().to(device)
            batch_x = aggregation(batch_time, batch_static)
            outputs = model_tor(batch_x)
            loss = criterion.loss(outputs, batch_y)
            loss.backward()
            train_loss.append(loss.item())
            model_optim.step()


    list_idx = idx_valid
    x_time, x_static, _ = gen_data_AI_batch_2_model(df_new, list_idx, L_all, L_out, L_ground, N_input, N_output, N_static, sliding_step, list_time_series, list_output_time, list_static, real_scaler, static_scaler)
    x_time = x_time.reshape(len(list_idx),-1,L_all, N_input+N_output)
    x_static = x_static.reshape(len(list_idx),-1, 1, N_static)

    y_predict_all = np.zeros((len(x_time), L_ground+L_all, N_output))

    for j in range(int(L_ground/L_out)+1):
      x_time[:,j,:L_all, N_input:N_input+N_output] = y_predict_all[:,L_out*j:L_all+L_out*j,:]
      xt = torch.from_numpy(x_time[:,j,:,:]).float().to('cuda')
      xs = torch.from_numpy(x_static[:,j,:,:]).float().to('cuda')
      batch_x = aggregation(xt, xs)
      y_pred_all = model_tor(batch_x).to('cuda').cpu().detach().numpy()
      y_predict_all[:,L_in+L_out*j:L_all+L_out*j,:] = y_pred_all[:,:,:,3]

    y_predict_all = target_scaler.inverse_transform(y_predict_all.reshape(-1, N_output))
    y_predict_all = y_predict_all.reshape(len(x_time), -1, N_output)


    dict_MAE = {k:[] for k in list_output_time}
    dict_R2 = {k:[] for k in list_output_time}

    for i_s in range(len(list_output_time)):
      for i in range(len(list_idx)):
        calMAE = MAE(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_MAE[list_output_time[i_s]].append(calMAE)
        calR2 = R2(df_new[list_output_time[i_s]][list_idx[i]], y_predict_all[i, L_all:L_all+L_ground,i_s])
        dict_R2[list_output_time[i_s]].append(calR2)

    mean_MAE = [np.mean(dict_MAE[k]) for k in dict_MAE]
    mean_R2 = [np.mean(dict_R2[k]) for k in dict_R2]

    return mean_MAE[-1]


In [ ]:
list_ex = ['ex1', 'ex2', 'ex3', 'ex4', 'ex5',
           '3Dex1', '3Dex2', '3Dex3', '3Dex4', '3Dex5']
dict_result = {}
for ex_name in list_ex:
  idx_train = np.random.choice(240, size=200, replace=False)
  idx_valid = [i for i in range(240) if i not in idx_train]
  study = optuna.create_study(direction='minimize')
  study.optimize(one_function, n_trials=200)
  dict_result[ex_name] = {"Best trial": study.best_trial.number,
                          "Best accuracy": study.best_trial.value,
                          "Best hyperparameters": study.best_params}



In [ ]:
dict_result = {}
for ex_name in list_ex:
  idx_train = np.random.choice(240, size=200, replace=False)
  idx_valid = [i for i in range(240) if i not in idx_train]
  study = optuna.create_study(direction='minimize')
  study.optimize(one_function, n_trials=30)
  dict_result[ex_name] = {"Best trial": study.best_trial.number, "Best accuracy": study.best_trial.value, "Best hyperparameters": study.best_params}



[I 2024-11-15 09:20:14,284] A new study created in memory with name: no-name-55de0349-e77a-4cc2-9f42-871be14ccb2c


3Dex4 250


[I 2024-11-15 09:21:55,444] Trial 0 finished with value: 0.004574549299411824 and parameters: {'d_model': 64, 'e_layers': 5, 'L_in': 250, 'L_out': 1000, 'dropout': 0.3}. Best is trial 0 with value: 0.004574549299411824.


3Dex4 250


[I 2024-11-15 09:22:56,975] Trial 1 finished with value: 0.004092870296847945 and parameters: {'d_model': 64, 'e_layers': 3, 'L_in': 50, 'L_out': 500, 'dropout': 0.3}. Best is trial 1 with value: 0.004092870296847945.


...

[I 2024-11-15 12:09:15,134] Trial 27 finished with value: 0.005037820347664129 and parameters: {'d_model': 256, 'e_layers': 2, 'L_in': 150, 'L_out': 300, 'dropout': 0.1}. Best is trial 12 with value: 0.003962739715981857.


3Dex9 254


[I 2024-11-15 12:11:08,105] Trial 28 finished with value: 0.0041292268786511125 and parameters: {'d_model': 256, 'e_layers': 4, 'L_in': 150, 'L_out': 500, 'dropout': 0.2}. Best is trial 12 with value: 0.003962739715981857.


3Dex9 254


[I 2024-11-15 12:12:37,924] Trial 29 finished with value: 0.004598227297153017 and parameters: {'d_model': 64, 'e_layers': 4, 'L_in': 500, 'L_out': 500, 'dropout': 0.1}. Best is trial 12 with value: 0.003962739715981857.
